# 20 Newsgroups text classification with pre-trained word embeddings

In this notebook, we'll use pre-trained [GloVe word embeddings](http://nlp.stanford.edu/projects/glove/) for text classification using TensorFlow 2.0 / Keras. This notebook is largely based on the blog post [Using pre-trained word embeddings in a Keras model](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) by François Chollet.

**Note that using a GPU with this notebook is highly recommended.**

First, the needed imports.

In [ ]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical, plot_model

from sklearn.metrics import confusion_matrix

import os, datetime
import sys

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

print('Using Tensorflow version:', tf.__version__,
      'Keras version:', tf.keras.__version__,
      'backend:', tf.keras.backend.backend())

If we are using TensorFlow as the backend, we can use TensorBoard to visualize our progress during training.

## GloVe word embeddings

Let's begin by loading a datafile containing pre-trained word embeddings from [Pouta Object Storage](https://research.csc.fi/pouta-object-storage).  The datafile contains 100-dimensional embeddings for 400,000 English words.  

In [ ]:
#!wget -nc https://object.pouta.csc.fi/swift/v1/AUTH_dac/mldata/glove6b100dtxt.zip
#!unzip -u glove6b100dtxt.zip
#GLOVE_DIR = "."

GLOVE_DIR = "/media/data/glove.6B"

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

print('Examples of embeddings:')
for w in ['some', 'random', 'words']:
    print(w, embeddings_index[w])

## 20 Newsgroups data set

Next we'll load the [20 Newsgroups](http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html) data set. 

The dataset contains 20000 messages collected from 20 different Usenet newsgroups (1000 messages from each group):

|[]()|[]()|[]()|[]()|
| --- | --- |--- | --- |
| alt.atheism           | soc.religion.christian   | comp.windows.x     | sci.crypt |               
| talk.politics.guns    | comp.sys.ibm.pc.hardware | rec.autos          | sci.electronics |              
| talk.politics.mideast | comp.graphics            | rec.motorcycles    | sci.space |                   
| talk.politics.misc    | comp.os.ms-windows.misc  | rec.sport.baseball | sci.med |                     
| talk.religion.misc    | comp.sys.mac.hardware    | rec.sport.hockey   | misc.forsale |

In [ ]:
#!wget -nc https://object.pouta.csc.fi/swift/v1/AUTH_dac/mldata/news20.tar.gz
#!tar -x --skip-old-files -f news20.tar.gz
#TEXT_DATA_DIR = "./20_newsgroup"

TEXT_DATA_DIR = "/media/data/20_newsgroup"

print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))

First message and its label:

In [ ]:
print(texts[0])
print('label:', labels[0], labels_index)

### Vectorization

Vectorize the text samples into a 2D integer tensor.

In [ ]:
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000 

tokenizer = text.Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = sequence.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

### TF Datasets

Let's now define our [TF `Dataset`s](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#class_dataset) for training, validation, and test data. 

In [ ]:
VALIDATION_SET, TEST_SET = 1000, 4000
BATCH_SIZE = 128 

dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.shuffle(20000, reshuffle_each_iteration=False)

train_dataset = dataset.skip(VALIDATION_SET+TEST_SET)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

validation_dataset = dataset.skip(TEST_SET).take(VALIDATION_SET)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True)

test_dataset = dataset.take(TEST_SET)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=False)

### Pretrained embedding matrix

As the last step in data preparation, we construct the GloVe embedding matrix:

In [ ]:
print('Preparing embedding matrix.')

num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_dim = 100

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
print('Shape of embedding matrix:', embedding_matrix.shape)

## 1-D CNN

### Initialization


In [ ]:
print('Build model...')
inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False)(inputs)

x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

outputs = layers.Dense(20, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs,
                    name="cnn_model")
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print(model.summary())

In [ ]:
plot_model(model, 'tf2-20ng-cnn.png', show_shapes=True)

### Learning

In [ ]:
logdir = os.path.join(os.getcwd(), "logs",
                      "20ng-cnn-"+datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
print('TensorBoard log directory:', logdir)
os.makedirs(logdir)
callbacks = [TensorBoard(log_dir=logdir)]

In [ ]:
%%time
epochs = 10

history = model.fit(train_dataset, epochs=epochs,
                    validation_data=validation_dataset,
                    verbose=2, callbacks=callbacks)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3))

ax1.plot(history.epoch,history.history['loss'], label='training')
ax1.plot(history.epoch,history.history['val_loss'], label='validation')
ax1.set_title('loss')
ax1.set_xlabel('epoch')
ax1.legend(loc='best')

ax2.plot(history.epoch,history.history['accuracy'], label='training')
ax2.plot(history.epoch,history.history['val_accuracy'], label='validation')
ax2.set_title('accuracy')
ax2.set_xlabel('epoch')
ax2.legend(loc='best');

### Inference

We evaluate the model using the test set. If accuracy on the test set is notably worse than with the training set, the model has likely overfitted to the training samples.

In [ ]:
%%time

test_scores = model.evaluate(test_dataset, verbose=2)
print("Test set %s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))

We can also look at classification accuracies separately for each newsgroup, and compute a confusion matrix to see which newsgroups get mixed the most:

In [ ]:
test_predictions = model.predict(test_dataset)
test_gt = np.concatenate([y for x, y in test_dataset], axis=0)

cm=confusion_matrix(np.argmax(test_gt, axis=1),
                    np.argmax(test_predictions, axis=1),
                    labels=list(range(20)))

print('Classification accuracy for each newsgroup:'); print()
labels = [l[0] for l in sorted(labels_index.items(), key=lambda x: x[1])]
for i,j in enumerate(cm.diagonal()/cm.sum(axis=1)): print("%s: %.4f" % (labels[i].ljust(26), j))
print()

print('Confusion matrix (rows: true newsgroup; columns: predicted newsgroup):'); print()
np.set_printoptions(linewidth=9999)
print(cm); print()

plt.figure(figsize=(10,10))
plt.imshow(cm, cmap="gray", interpolation="none")
plt.title('Confusion matrix (rows: true newsgroup; columns: predicted newsgroup)')
plt.grid(None)
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=90)
plt.yticks(tick_marks, labels);

## LSTM

### Initialization

In [ ]:
print('Build model...')
inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False)(inputs)

x = layers.LSTM(128)(x)
x = layers.Dense(128, activation='relu')(x)

outputs = layers.Dense(20, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs,
                    name="lstm_model")
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print(model.summary())

In [ ]:
plot_model(model, 'tf2-20ng-rnn.png', show_shapes=True)

### Learning

In [ ]:
logdir = os.path.join(os.getcwd(), "logs",
                      "20ng-rnn-"+datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
print('TensorBoard log directory:', logdir)
os.makedirs(logdir)
callbacks = [TensorBoard(log_dir=logdir)]

In [ ]:
%%time
epochs = 10

history = model.fit(train_dataset, epochs=epochs,
                    validation_data=validation_dataset,
                    verbose=2, callbacks=callbacks)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3))

ax1.plot(history.epoch,history.history['loss'], label='training')
ax1.plot(history.epoch,history.history['val_loss'], label='validation')
ax1.set_title('loss')
ax1.set_xlabel('epoch')
ax1.legend(loc='best')

ax2.plot(history.epoch,history.history['accuracy'], label='training')
ax2.plot(history.epoch,history.history['val_accuracy'], label='validation')
ax2.set_title('accuracy')
ax2.set_xlabel('epoch')
ax2.legend(loc='best');

### Inference

In [ ]:
%%time

test_scores = model.evaluate(test_dataset, verbose=2)
print("Test set %s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))

In [ ]:
test_predictions = model.predict(test_dataset)
test_gt = np.concatenate([y for x, y in test_dataset], axis=0)

cm=confusion_matrix(np.argmax(test_gt, axis=1),
                    np.argmax(test_predictions, axis=1),
                    labels=list(range(20)))

print('Classification accuracy for each newsgroup:'); print()
labels = [l[0] for l in sorted(labels_index.items(), key=lambda x: x[1])]
for i,j in enumerate(cm.diagonal()/cm.sum(axis=1)): print("%s: %.4f" % (labels[i].ljust(26), j))
print()

print('Confusion matrix (rows: true newsgroup; columns: predicted newsgroup):'); print()
np.set_printoptions(linewidth=9999)
print(cm); print()

plt.figure(figsize=(10,10))
plt.imshow(cm, cmap="gray", interpolation="none")
plt.title('Confusion matrix (rows: true newsgroup; columns: predicted newsgroup)')
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=90)
plt.yticks(tick_marks, labels);